
Title: "Online Shopping data cleaning and interpolation" 
Author : "Gabriel E Mancillas Gallardo
date: "2023-12-08" 


Here is a rundown of what my portion of work might include: 

Understanding the Dataset:
Familiarize yourself with the dataset by identifying the number of features (columns), records (rows), and the type of data (numerical, categorical, boolean).
Determine the significance of each feature and its potential role in any analysis or predictive modeling.

Data Importing:
Import the dataset into an appropriate data analysis tool (like Python with pandas, R, etc.).
Verify that the data has been imported correctly by checking for any discrepancies in the number of rows and columns.

Data Cleaning:
Check for and handle missing values, even though initial inspection suggests there are none. Sometimes missing values are hidden as placeholders (like zeros, negative values, or strings such as 'unknown', 'n/a').
Detect and correct errors or inconsistencies in data, such as typos, incorrect entries, or mislabeled classes.
Remove duplicate records to prevent skewed analysis.

Data Transformation:
Normalize or scale numerical features to ensure that no variable dominates others because of its scale, which is important for models like k-nearest neighbors (KNN) or gradient descent-based algorithms.
Encode categorical variables using techniques like one-hot encoding or label encoding, so that they can be used in mathematical models.
Aggregate data if needed, for instance, summarizing daily data into monthly data if the analysis calls for it.

Feature Engineering:
Create new features that could be important for the analysis or predictive modeling. For instance, from the Month column, you could derive a feature that represents the season.
Discretize continuous features if it makes sense for the analysis. For example, you could categorize Age into groups like 'Young', 'Middle-Aged', and 'Senior'.

Data Reduction:
Reduce dimensionality if the dataset is very large or contains many features that are not contributing to the analysis. Techniques like Principal Component Analysis (PCA) can be useful here.
Perform feature selection to keep the most relevant features, which could involve domain knowledge, statistical tests, or model-based selection.

Ensuring Data Quality:
Validate the quality of the data to ensure accuracy, completeness, and reliability.
Document any assumptions or decisions made during the pre-processing steps.



In [31]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import sklearn
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.metrics import roc_auc_score, roc_curve


working_directory = os.getcwd()
print(working_directory)

path = working_directory + '/online_shoppers_intentions.csv' 
df = pd.read_csv(os.path.join(os.getcwd(),'online_shoppers_intentions.csv'))

df = pd.read_csv(path) 

/Users/gabrielmancillas


In [32]:
df_info = df.info()
df_head = df.head()
df_describe = df.describe()
df_nunique = {column: data[column].nunique() for column in data.columns}

df_info, df_head, df_describe, df_nunique

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12202 non-null  float64
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12195 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12207 non-null  float64
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

(None,
    Administrative  Administrative_Duration  Informational  \
 0               0                      0.0            0.0   
 1               0                      0.0            0.0   
 2               0                      0.0            0.0   
 3               0                      0.0            0.0   
 4               0                      0.0            0.0   
 
    Informational_Duration  ProductRelated  ProductRelated_Duration  \
 0                     0.0               1                 0.000000   
 1                     0.0               2                64.000000   
 2                     0.0               1                 0.000000   
 3                     0.0               2                 2.666667   
 4                     0.0              10               627.500000   
 
    BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
 0         0.20       0.20         0.0         0.0   Feb               1.0   
 1         0.00       0.10         

# DataFrame Information:

The dataset is contained in a DataFrame, which is a tabular data structure with labeled axes (rows and columns). It has 12,330 entries (rows) and 18 columns. Each column's name, number of non-null values, and data type (int64, float64, bool, object) are listed. There's a usage of 1.4+ MB memory by this DataFrame. First Few Rows (Head of the DataFrame):

The head() method output shows the first five rows of the DataFrame. Columns include various numerical and categorical features such as Administrative, Administrative_Duration, Informational, Informational_Duration, ProductRelated, ProductRelated_Duration, BounceRates, ExitRates, PageValues, SpecialDay, Month, OperatingSystems, Browser, Region, TrafficType, VisitorType, Weekend, and Revenue. Descriptive Statistics:

The describe() method provides a statistical summary of the numerical columns. This includes count, mean, standard deviation, minimum, 25th percentile, median (50th percentile), 75th percentile, and maximum values for columns like Administrative_Duration, Informational_Duration, ProductRelated_Duration, BounceRates, ExitRates, and PageValues. Unique Value Counts:

The nunique() method shows the count of unique values for each column, which helps to understand the distribution of categorical data. For example, it shows how many different types of browsers or operating systems are represented in the dataset.
***

In [33]:
df['Month'] = df['Month'].astype(str)

df['Month'] = df['Month'].replace('June', 'Jun')

# Convert 'Month' column to a categorical type with levels being the abbreviated month names
df['Month'] = pd.Categorical(df['Month'], categories=[
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
])

# Display the unique values in 'Month' column to verify the changes
df['Month'].unique(), df.dtypes['Month']

(['Feb', 'Mar', 'May', 'Oct', 'Jun', 'Jul', 'Aug', 'Nov', 'Sep', 'Dec']
 Categories (12, object): ['Jan', 'Feb', 'Mar', 'Apr', ..., 'Sep', 'Oct', 'Nov', 'Dec'],
 CategoricalDtype(categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
                   'Sep', 'Oct', 'Nov', 'Dec'],
 , ordered=False, categories_dtype=object))

In [34]:
# Count the number of duplicate rows
number_duplicate = df.duplicated().sum()

# Remove the duplicate rows from the dataframe
df = df[~df.duplicated()]

# Display the structure of the dataframe using 'info()' which is similar to 'str()' in R
df.info(), number_duplicate

<class 'pandas.core.frame.DataFrame'>
Index: 12209 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Administrative           12209 non-null  int64   
 1   Administrative_Duration  12209 non-null  float64 
 2   Informational            12081 non-null  float64 
 3   Informational_Duration   12209 non-null  float64 
 4   ProductRelated           12209 non-null  int64   
 5   ProductRelated_Duration  12209 non-null  float64 
 6   BounceRates              12209 non-null  float64 
 7   ExitRates                12209 non-null  float64 
 8   PageValues               12074 non-null  float64 
 9   SpecialDay               12209 non-null  float64 
 10  Month                    12209 non-null  category
 11  OperatingSystems         12086 non-null  float64 
 12  Browser                  12209 non-null  int64   
 13  Region                   12209 non-null  int64   
 14  TrafficType

(None, 121)

In [35]:
# Check for any missing values in the dataframe
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Informational       128
PageValues          135
OperatingSystems    123
dtype: int64

In [93]:
# Remove rows with any missing values from the dataframe
df.dropna(inplace=True)

# Verify that there are no missing values left
missing_values_cleaned = df_cleaned.isnull().sum()
df_cleaned.info(), missing_values_cleaned[missing_values_cleaned > 0]

<class 'pandas.core.frame.DataFrame'>
Index: 11827 entries, 0 to 12208
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Unnamed: 0               11827 non-null  int64   
 1   Administrative           11827 non-null  int64   
 2   Administrative_Duration  11827 non-null  float64 
 3   Informational            11827 non-null  float64 
 4   Informational_Duration   11827 non-null  float64 
 5   ProductRelated           11827 non-null  int64   
 6   ProductRelated_Duration  11827 non-null  float64 
 7   BounceRates              11827 non-null  float64 
 8   ExitRates                11827 non-null  float64 
 9   PageValues               11827 non-null  float64 
 10  SpecialDay               11827 non-null  float64 
 11  Month                    11827 non-null  category
 12  OperatingSystems         11827 non-null  float64 
 13  Browser                  11827 non-null  int64   
 14  Region     

(None, Series([], dtype: int64))

#### After conducting a thorough analysis of the data, we closely scrutinized a specific column for any unwanted horizontal lines that could have impeded its interpretation. We meticulously checked each value in the column and, upon careful inspection, determined that there were no such lines present. 

#### Furthermore, we removed any empty or null values from the dataset to ensure that it was free from any incomplete or missing information. We did this by carefully reviewing each entry and identifying any instances where data was either absent or unknown. After identifying the errors, we removed them from the dataset to ensure its overall accuracy and completeness. In addition, we reviewed each entry in the dataset to eliminate any duplicate data. 

In [94]:
for key in df.keys():
    print(key)
    print(np.unique(df[key]))
    print("===============")

Unnamed: 0
[    0     1     2 ... 12327 12328 12329]
Administrative
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 26 27]
Administrative_Duration
[0.00000000e+00 1.33333333e+00 2.00000000e+00 ... 2.65731806e+03
 2.72050000e+03 3.39875000e+03]
Informational
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 16. 24.]
Informational_Duration
[0.00000000e+00 1.00000000e+00 1.50000000e+00 ... 2.25203333e+03
 2.25691667e+03 2.54937500e+03]
ProductRelated
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 1

Notes:

-3 different types of time duration, administrative,product related and informational, detailed in readme, represents different parts of the website which correspond to the given names.

-Revenue is just boolean, doesn't contain dollars spent , number of purchases etc

Data categories:

AdministrativeDuration-Continuous
Administrative-Ordinal
ProductRelatedDuration- continuous
Information-Ordinal
InformationDuration - continuous
Product- Ordinal
ProductRelatedDuration- continuous
Bounce/Exit rates- conituous
Month - Ordinal
OperatingSystems - Nominal
Region - Nominal
Traffic Type - Nominal
Visitor type - Nominal
Weekend Boolean
Revenue Boolean

* We check for duplicate for rows counts - there is non. 

In [95]:
duplicate_rows_count = df.duplicated().sum()

In [109]:
print(duplicate_rows_count)

0


#### I checked the dataset and found that there are no missing values (Null). This indicates that the data is complete and reliable. Moreover, the information is equally distributed across the dataset which is a good sign.

In [116]:
columns_to_drop=["level_0","index","Unnamed: 0"]

In [117]:
df.to_csv('cleaned_online_shopping_data.csv', index=False)

The END OF OUR CLEANING. 